In [1]:
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout

d:\Poli\MIniconda\envs\tf_flow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\Poli\MIniconda\envs\tf_flow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\Poli\MIniconda\envs\tf_flow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\Poli\MIniconda\envs\tf_flow\lib\site-packages\tensorflow\python\framework\dtypes.py:5

O dataset foi gerado a partir da base de dados DREAMER:
S. Katsigiannis, N. Ramzan, “DREAMER: A Database for Emotion Recognition Through EEG and ECG
Signals from Wireless Low-cost Off-the-Shelf Devices,” IEEE Journal of Biomedical and Health
Informatics, 2017. In press. doi: 10.1109/JBHI.2017.2688239

In [ ]:
#Carregando o modelo
df = pd.read_csv('dreamer_preprocessed_features.csv')
df.head()

#A coluna gender deve ser convertida para valores numéricos (embora sua utilização se tornou prejudicial para a acurácia)
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df.head()

#Gerando input e output
colunas_para_remover = [ 'trial', 'participant','valence', 'arousal', 'dominance']
X = df.drop(columns=colunas_para_remover).copy()

output_features = ['valence', 'arousal', 'dominance']
y = df[output_features]

print(f"\nNúmero de características de entrada: {X.shape[1]}")
print("Características de entrada selecionadas:", X.columns.tolist())
print(f"\nNúmero de neurônios de saída: {y.shape[1]}")
print("Alvos de saída:", y.columns.tolist())


Número de características de entrada: 30
Características de entrada selecionadas: ['age', 'gender', 'eeg_mean_AF3', 'eeg_var_AF3', 'eeg_mean_F7', 'eeg_var_F7', 'eeg_mean_F3', 'eeg_var_F3', 'eeg_mean_FC5', 'eeg_var_FC5', 'eeg_mean_T7', 'eeg_var_T7', 'eeg_mean_P7', 'eeg_var_P7', 'eeg_mean_O1', 'eeg_var_O1', 'eeg_mean_O2', 'eeg_var_O2', 'eeg_mean_P8', 'eeg_var_P8', 'eeg_mean_T8', 'eeg_var_T8', 'eeg_mean_FC6', 'eeg_var_FC6', 'eeg_mean_F4', 'eeg_var_F4', 'eeg_mean_F8', 'eeg_var_F8', 'eeg_mean_AF4', 'eeg_var_AF4']

Número de neurônios de saída: 3
Alvos de saída: ['valence', 'arousal', 'dominance']


In [ ]:
#Splitando o dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalonamento dos dados de entrada.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # Importante: usar o mesmo scaler treinado nos dados de treino.

In [ ]:
#Criando o modelo

model = Sequential(name="Emotion_Prediction_Model")
  
model.add(InputLayer(input_shape=(X_train_scaled.shape[1],), name="Input_Layer"))
model.add(Dense(64, activation='relu', name="Hidden_Layer_1"))
model.add(Dense(32, activation='relu', name="Hidden_Layer_2"))
model.add(Dense(3, activation='linear', name="Output_Layer"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error']) # MAE é mais fácil de interpretar que o MSE.

# Mostra um resumo da arquitetura do modelo.
print("\n--- Arquitetura da Rede Neural ---")
model.summary()

    
print("\n--- Iniciando o Treinamento ---")
# O treinamento é executado por 100 épocas.
# validation_data é usado para monitorar o desempenho do modelo em dados não vistos a cada época.
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=8,
    validation_data=(X_test_scaled, y_test),
    verbose=0  # Use verbose=1 para ver o progresso de cada época
)
print("Treinamento concluído.")


--- Arquitetura da Rede Neural ---
Model: "Emotion_Prediction_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Hidden_Layer_1 (Dense)       (None, 64)                1984      
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 32)                2080      
_________________________________________________________________
Output_Layer (Dense)         (None, 3)                 99        
Total params: 4,163
Trainable params: 4,163
Non-trainable params: 0
_________________________________________________________________

--- Iniciando o Treinamento ---
Treinamento concluído.


In [ ]:
print("\n--- Avaliação do Modelo no Conjunto de Teste ---")
loss, mae = model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"  -> Perda (Mean Squared Error - MSE): {loss:.4f}")
print(f"  -> Erro Médio Absoluto (Mean Absolute Error - MAE): {mae:.4f} \n ")




y_pred = model.predict(X_test_scaled)
df_reais = y_test.reset_index(drop=True)
df_previstos = pd.DataFrame(y_pred, columns=['Valence_Prevista', 'Arousal_Previsto', 'Dominance_Previsto'])
df_comparacao = pd.concat([df_reais, df_previstos], axis=1).round(2)

# 6. Renomeando as colunas originais 
df_comparacao.rename(columns={
    'valence': 'Valence_Real',
    'arousal': 'Arousal_Real',
    'dominance': 'Dominance_Real'
}, inplace=True)

#Reorganizando
df_comparacao = df_comparacao[[
    'Valence_Real', 'Valence_Prevista',
    'Arousal_Real', 'Arousal_Previsto',
    'Dominance_Real', 'Dominance_Previsto']]

#Printando a tabela de comparação completa
print(df_comparacao.to_string())


--- Avaliação do Modelo no Conjunto de Teste ---
  -> Perda (Mean Squared Error - MSE): 1.0950
  -> Erro Médio Absoluto (Mean Absolute Error - MAE): 0.7649 
 

--- Comparando cada Previsão com o Valor Real ---
   Valence_Real  Valence_Prevista  Arousal_Real  Arousal_Previsto  Dominance_Real  Dominance_Previsto
0             2              3.30             2              2.33               2                2.26
1             4              4.20             1              2.78               2                2.18
2             4              6.49             3              4.11               2                3.67
3             3              3.29             2              1.78               2                1.26
4             3              2.42             2              1.95               2                1.73
